## Objetivo

Me interesa saber la distribución de los costos de productos adquiridos por la secretaría de salud. Esto es, ¿Cuánto de más o cuánto de menos se ha estado pagando? ¿Podemos detectar corrupción? ¿Comportamiento por distribuidores?

## Obtener la data

No logre encontrar los datos abiertos que alimentan: http://compras.imss.gob.mx/?P=search_alt Voy a proceder a scrappearlo.

Alto, están las compras consolidadas: Según HHealth (Gracias!) no son todas las compras. pero está chido para hacerme de un diccionario.
http://www.imss.gob.mx/compraconsolidada2016

In [81]:
import requests as r
import os
import json
from bs4 import BeautifulSoup

In [76]:
def products(field,page):
    result = os.popen("curl 'http://buscador.compras.imss.gob.mx/index.php' -H 'Pragma: no-cache' -H 'Origin: http://buscador.compras.imss.gob.mx' -H 'Accept-Encoding: gzip, deflate' -H 'Accept-Language: es-ES,es;q=0.8,en;q=0.6' -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36' -H 'Content-Type: application/x-www-form-urlencoded' -H 'Accept: */*' -H 'Cache-Control: no-cache' -H 'X-Requested-With: WAJAF::Ajax - WebAbility(r) v5' -H 'Cookie: __utma=37030712.222899114.1488655887.1488656746.1488656746.1; __utmb=37030712.1.10.1488656746; __utmc=37030712; __utmz=37030712.1488656746.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __asc=9819a67f15a9ace2b0599c2b83a; __auc=9819a67f15a9ace2b0599c2b83a; _ga=GA1.3.222899114.1488655887' -H 'Connection: keep-alive' -H 'Referer: http://buscador.compras.imss.gob.mx/wrap/index.html' --data '&type=compras&message=X&filtered=1&descripcion="+field+"&proveedor=&numcompra=&delegacion=values%3D&fecha=min%3D%3Bmax%3D&procedimiento=values%3D&exact=false&numperpage=20&page=" + str(page)+"&order=fecha%20desc' --compressed").read()
    return(json.loads(result))

In [104]:
first_page  =search_page(" ",1)

In [107]:
first_page["result"].keys()

dict_keys(['near', 'page', 'exact', 'time', 'quantity', 'id', 'data', 'numperpage', 'nquery'])

In [109]:
int(first_page["result"]["quantity"])

2136708

In [111]:
first_page["result"]["data"][0]

{'clave': '12086929',
 'delegacion': None,
 'descripcion': '176',
 'fecha': '',
 'monto': '0.00',
 'numcompra': None,
 'procedimiento': None,
 'proveedor': None}

In [98]:
def transaction(transaction_number):
    transaction_page = "http://compras.imss.gob.mx/?P=imsscomprofich&f="+ str(transaction_number)
    content = r.get(transaction_page)
    soup = BeautifulSoup(content.content)
    local_data = {}
    for span in soup.findAll("span", {"class":"txtdesccaja"}):
        split_span = span.text.replace("\n","").strip().split(":")
        name = split_span[0]
        content = " ".join(split_span[1:])
        local_data[name]=content
    return(local_data)

In [100]:
transaction(12086888)

{'# Factura': ' D17159',
 'COMERCIAL HOSPITALARIA S.A DE C.V.': '',
 'Cantidad recibida': ' 30',
 'Cantidad solicitada': ' 30',
 'Delegación del IMSS': ' Delegación Sonora',
 'Descripción': ' TIRA REACTIVA PARA DETERMINACION SEMICUANTITATIVA DE GLUCOSA EN SANGRE CON   LIMITES DE DETECCION QUE VAN DE 20 A 800 MG/DL. FRASCO CON 50 TIRAS. TA.',
 'Estado de la República': ' Sonora',
 'Fecha factura': ' 17/02/17',
 'IVA': ' 16',
 'No. Procedimiento': ' 230',
 'Precio': ' 294.00',
 'Procedimiento de compra': 'Compra emergente en unidades de servicio',
 'RFC': ' CHO011105F23 [Ver ficha del proveedor]',
 'Subprocedimiento de compra': ' N/A',
 'Unidad': ' TRA'}

In [101]:
transaction(12086887)

{'# Factura': ' D17094',
 'COMERCIAL HOSPITALARIA S.A DE C.V.': '',
 'Cantidad recibida': ' 15',
 'Cantidad solicitada': ' 15',
 'Delegación del IMSS': ' Delegación Sonora',
 'Descripción': ' RIFAMPICINA SUSPENSION ORAL CADA 5 ML CONTIENEN  RIFAMPICINA 100 MG ENVASE CON   120 ML Y DOSIFICADOR.',
 'Estado de la República': ' Sonora',
 'Fecha factura': ' 13/02/17',
 'IVA': ' No contiene datos',
 'No. Procedimiento': ' 171',
 'Precio': ' 438.90',
 'Procedimiento de compra': 'Compra emergente en unidades de servicio',
 'RFC': ' CHO011105F23 [Ver ficha del proveedor]',
 'Subprocedimiento de compra': ' N/A',
 'Unidad': ' ENV'}

In [110]:
def get_all_data():
    first_page = search_page(" ",1)
    data = first_page["result"]["data"]
    hits = int(first_page["result"]["quantity"])
    pages = floor(hits/20) + 1
    i = 1
    while i<=pages:
        try:
            page_data = search_page(" ",i)
            data = data + page_data["result"]["data"]
            i = i +1
        except:
            None
    return(data)

In [65]:
import math
math.floor(int(results["result"]["quantity"])/20)+1

106836

In [67]:
results

{'result': {'data': [{'clave': '12086929',
    'delegacion': None,
    'descripcion': '176',
    'fecha': '',
    'monto': '0.00',
    'numcompra': None,
    'procedimiento': None,
    'proveedor': None},
   {'clave': '12086928',
    'delegacion': None,
    'descripcion': '125',
    'fecha': '',
    'monto': '0.00',
    'numcompra': None,
    'procedimiento': None,
    'proveedor': None},
   {'clave': '12086927',
    'delegacion': None,
    'descripcion': '9',
    'fecha': '',
    'monto': '0.00',
    'numcompra': None,
    'procedimiento': None,
    'proveedor': None},
   {'clave': '12086926',
    'delegacion': None,
    'descripcion': '90',
    'fecha': '',
    'monto': '0.00',
    'numcompra': None,
    'procedimiento': None,
    'proveedor': None},
   {'clave': '12086925',
    'delegacion': None,
    'descripcion': '96',
    'fecha': '',
    'monto': '0.00',
    'numcompra': None,
    'procedimiento': None,
    'proveedor': None},
   {'clave': '12086924',
    'delegacion': None,
  

In [74]:
results = search_page(" ",3)

In [75]:
results

{'result': {'data': [{'clave': '12086889',
    'delegacion': None,
    'descripcion': '196',
    'fecha': '',
    'monto': '0.00',
    'numcompra': None,
    'procedimiento': None,
    'proveedor': None},
   {'clave': '12086888',
    'delegacion': None,
    'descripcion': '230',
    'fecha': '',
    'monto': '0.00',
    'numcompra': None,
    'procedimiento': None,
    'proveedor': None},
   {'clave': '12086887',
    'delegacion': None,
    'descripcion': '171',
    'fecha': '',
    'monto': '0.00',
    'numcompra': None,
    'procedimiento': None,
    'proveedor': None},
   {'clave': '12086886',
    'delegacion': None,
    'descripcion': '231',
    'fecha': '',
    'monto': '0.00',
    'numcompra': None,
    'procedimiento': None,
    'proveedor': None},
   {'clave': '12086885',
    'delegacion': None,
    'descripcion': '224',
    'fecha': '',
    'monto': '0.00',
    'numcompra': None,
    'procedimiento': None,
    'proveedor': None},
   {'clave': '12086884',
    'delegacion': None